In [14]:
dataset_name = "plane-dataset" # Name of the Voxel51 Dataset to use

### Load the Voxel51 Dataset

In [22]:
#config
import fiftyone as fo
from fiftyone import ViewField as F
import os
import numpy as np
import matplotlib.pyplot as plt


dataset = fo.load_dataset(dataset_name)
dataset.reload()

### Browse the entire Voxel51 Dataset

In [38]:
dataset.reload()
session = fo.launch_app(dataset, auto=False)

Session launched. Run `session.show()` to open the App in a cell output.


### Browse a subsection of the Voxel51 Dataset

In [4]:
#view = dataset.exists("training")

# Veiw samples where there is no label
# view = dataset.match({"model": {"$exists": False, "$eq": None}})

# View samples with a tag
view = dataset.match(F("multi_class_detections.detections").length()>0)
#view = dataset.match_tags("train")

print(view)

session.view = view


Dataset:     plane-dataset
Media type:  image
Num samples: 11204
Tags:        ['capture-3-29', 'capture-3-30', 'capture-5-13', 'eval', 'model_eval', 'model_train', 'multi_class_eval', 'multi_class_train', 'train']
Sample fields:
    id:                                                                       fiftyone.core.fields.ObjectIdField
    filepath:                                                                 fiftyone.core.fields.StringField
    tags:                                                                     fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:                                                                 fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    external_id:                                                              fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    icao24:                                                                   fiftyone.core.

### Add Tags to Selected Samples
Use the browser to select samples and add tags to them

In [ ]:
# Create a view containing only the selected samples
selected_view = dataset.select(session.selected)
print(selected_view)
for sample in selected_view:
    sample.tags.append("training")
    sample.save() 

### Remove Tags to Selected Samples
Use the browser to select samples and remove tags from them

In [ ]:
# Create a view containing only the selected samples
selected_view = dataset.select(session.selected)
print(selected_view)
for sample in selected_view:
    sample.tags.remove("eval")
    sample.save() 

### Add a Tag to All Samples with a given Label Field

In [ ]:
view = dataset.exists("detections")
print(view)
for sample in view:
    sample.tags.append("training")
    sample.save() 

### Add a Tag to All Samples in a Dataset

In [ ]:
for sample in dataset:
    sample.tags.append("capture-5-13")
    sample.save() 

### Number of Unique Aircraft Bodies per Normalized Model

In [12]:

norm_models = dataset.distinct("norm_model.label")
for norm_model in norm_models:
    view = dataset.filter_labels("norm_model", (F("label") == norm_model)).select_fields("icao24")
    unique_aircraft = view.distinct("icao24.label")
    
    #print( unique_aircraft)
    total_predictions=0
    for icao24 in unique_aircraft:
        plane_view = dataset.filter_labels("icao24", (F("label") == icao24))

        for sample in plane_view:
            #print(sample["auto_aug_predict_tiled"])
            total_predictions = total_predictions + len(sample["auto_aug_predict_tiled"].detections)

    #print("{},{},{}".format(norm_model,len(unique_aircraft),total_predictions))     

737-400,1,46
737-700,16,769
737-800,24,1299
737-900,12,668
757-200,3,200
767-200,1,40
777-200,2,127
A300,2,108
A319,17,1132
A320,72,4051
A321,27,1506
Bombardier Challenger,10,488
CRJ200,1,44
CRJ550,2,111
CRJ700,5,207
CRJ900,9,501
Cessna Jet,17,900
EMB-135,8,295
EMB-505,6,217
ERJ-170,63,3484
Falcon,1,41
Gulfstream,8,334
Hawker 400,5,192
Hawker 800,4,212
HondaJet,1,45
King Air,1,53
LearJet,2,87
Learjet 45/60,3,118
Pilatus PC-12,4,287
Piper Cheyenne,1,71


In [64]:
import math
norm_models = dataset.distinct("norm_model.label")
for norm_model in norm_models:
    view = dataset.filter_labels("norm_model", (F("label") == norm_model)).select_fields("icao24")
    unique_aircraft = view.distinct("icao24.label")
    if len(unique_aircraft) > 1:
        print("{} Total: {} Trian: {} Test: {}".format(norm_model,len(unique_aircraft),math.floor(len(unique_aircraft)*.75),math.ceil(len(unique_aircraft)*.25)))     
   

737-700 Total: 16 Trian: 12 Test: 4
737-800 Total: 24 Trian: 18 Test: 6
737-900 Total: 12 Trian: 9 Test: 3
757-200 Total: 3 Trian: 2 Test: 1
777-200 Total: 2 Trian: 1 Test: 1
A300 Total: 2 Trian: 1 Test: 1
A319 Total: 17 Trian: 12 Test: 5
A320 Total: 72 Trian: 54 Test: 18
A321 Total: 27 Trian: 20 Test: 7
Bombardier Challenger Total: 10 Trian: 7 Test: 3
CRJ550 Total: 2 Trian: 1 Test: 1
CRJ700 Total: 5 Trian: 3 Test: 2
CRJ900 Total: 9 Trian: 6 Test: 3
Cessna Jet Total: 17 Trian: 12 Test: 5
EMB-135 Total: 8 Trian: 6 Test: 2
EMB-505 Total: 6 Trian: 4 Test: 2
ERJ-170 Total: 63 Trian: 47 Test: 16
Gulfstream Total: 8 Trian: 6 Test: 2
Hawker 400 Total: 5 Trian: 3 Test: 2
Hawker 800 Total: 4 Trian: 3 Test: 1
LearJet Total: 2 Trian: 1 Test: 1
Learjet 45/60 Total: 3 Trian: 2 Test: 1
Pilatus PC-12 Total: 4 Trian: 3 Test: 1


In [21]:
norm_models = dataset.distinct("norm_model.label")
for norm_model in norm_models:
    view = dataset.filter_labels("norm_model", (F("label") == norm_model)).select_fields("icao24")
    unique_aircraft = view.distinct("icao24.label")

    #print( unique_aircraft)
    total_predictions=0

    plane_view = dataset.filter_labels("icao24", (F("label") == unique_aircraft[0]))

    for sample in plane_view:
        #print(sample["auto_aug_predict_tiled"])
        total_predictions = total_predictions + len(sample["auto_aug_predict_tiled"].detections)

    print("{}: {}".format(norm_model,total_predictions))  

737-400: 46
737-700: 70
737-800: 36
737-900: 67
757-200: 46
767-200: 40
777-200: 58
A300: 49
A319: 44
A320: 50
A321: 62
Bombardier Challenger: 50
CRJ200: 44
CRJ550: 51
CRJ700: 68
CRJ900: 59
Cessna Jet: 66
EMB-135: 52
EMB-505: 66
ERJ-170: 108
Falcon: 41
Gulfstream: 69
Hawker 400: 44
Hawker 800: 65
HondaJet: 45
King Air: 53
LearJet: 64
Learjet 45/60: 54
Pilatus PC-12: 69
Piper Cheyenne: 71


In [26]:
norm_models = dataset.distinct("norm_model.label")
for norm_model in norm_models:
    view = dataset.match_tags("multi_class_eval").filter_labels("norm_model", (F("label") == norm_model)).match(F("auto_aug_predict_tiled.detections").length()>0)
    print("{}: {}".format(norm_model,len(view)))


737-400: 0
737-700: 195
737-800: 322
737-900: 113
757-200: 29
767-200: 0
777-200: 69
A300: 0
A319: 245
A320: 668
A321: 235
Bombardier Challenger: 137
CRJ200: 0
CRJ550: 0
CRJ700: 46
CRJ900: 49
Cessna Jet: 221
EMB-135: 102
EMB-505: 39
ERJ-170: 751
Falcon: 0
Gulfstream: 32
Hawker 400: 82
Hawker 800: 61
HondaJet: 0
King Air: 0
LearJet: 23
Learjet 45/60: 28
Pilatus PC-12: 0
Piper Cheyenne: 0


In [33]:
norm_models = dataset.match_tags("multi_class_eval").distinct("norm_model.label")
print("{}".format(len(norm_models)))

20


In [40]:
norm_models = dataset.distinct("norm_model.label")
for norm_model in norm_models:
    view = dataset.match_tags("multi_class_eval").filter_labels("norm_model", (F("label") == norm_model)).match(F("auto_aug_predict_tiled.detections").length()>0)
    eval_len = len(view)
    view = dataset.match_tags("multi_class_train").filter_labels("norm_model", (F("label") == norm_model)).match(F("auto_aug_predict_tiled.detections").length()>0)
    train_len = len(view)
    print("{}: train - {} / eval - {}".format(norm_model,train_len,eval_len))

737-400: train - 0 / eval - 0
737-700: train - 573 / eval - 195
737-800: train - 976 / eval - 322
737-900: train - 554 / eval - 113
757-200: train - 110 / eval - 90
767-200: train - 0 / eval - 0
777-200: train - 58 / eval - 69
A300: train - 49 / eval - 59
A319: train - 887 / eval - 245
A320: train - 3045 / eval - 1004
A321: train - 1209 / eval - 295
Bombardier Challenger: train - 351 / eval - 137
CRJ200: train - 0 / eval - 0
CRJ550: train - 51 / eval - 59
CRJ700: train - 161 / eval - 46
CRJ900: train - 323 / eval - 177
Cessna Jet: train - 630 / eval - 269
EMB-135: train - 193 / eval - 102
EMB-505: train - 178 / eval - 39
ERJ-170: train - 2719 / eval - 751
Falcon: train - 0 / eval - 0
Gulfstream: train - 302 / eval - 32
Hawker 400: train - 110 / eval - 82
Hawker 800: train - 147 / eval - 61
HondaJet: train - 0 / eval - 0
King Air: train - 0 / eval - 0
LearJet: train - 64 / eval - 23
Learjet 45/60: train - 90 / eval - 28
Pilatus PC-12: train - 222 / eval - 65
Piper Cheyenne: train - 0 / 

In [39]:
import math
norm_models = dataset.distinct("norm_model.label")
for norm_model in norm_models:
    view = dataset.filter_labels("norm_model", (F("label") == norm_model)).select_fields("icao24").shuffle()
    unique_aircraft = view.distinct("icao24.label")
    if len(unique_aircraft) > 1:
        train_aircraft = unique_aircraft[:math.floor(len(unique_aircraft)*.75)]
        total_detect = 0
        for aircraft in train_aircraft:
            view = dataset.filter_labels("icao24", (F("label") == aircraft)).match(F("multi_class_detections.detections").length()>0)
            total_detect = len(view) + total_detect
        actual_train = total_detect
        eval_aircraft = unique_aircraft[math.floor(len(unique_aircraft)*.75):]
        total_detect = 0
        for aircraft in eval_aircraft:
            view = dataset.filter_labels("icao24", (F("label") == aircraft)).match(F("multi_class_detections.detections").length()>0)
            total_detect = len(view) + total_detect
        actual_eval = total_detect
        print("{} Total: {} Train: {} aircraft, {} images /  Eval: {} aircraft, {} images".format(norm_model,len(unique_aircraft),len(train_aircraft),actual_train,len(eval_aircraft), actual_eval))     


737-700 Total: 16 Train: 12 aircraft, 573 images /  Eval: 4 aircraft, 195 images
737-800 Total: 24 Train: 18 aircraft, 976 images /  Eval: 6 aircraft, 322 images
737-900 Total: 12 Train: 9 aircraft, 554 images /  Eval: 3 aircraft, 113 images
757-200 Total: 3 Train: 2 aircraft, 110 images /  Eval: 1 aircraft, 90 images
777-200 Total: 2 Train: 1 aircraft, 58 images /  Eval: 1 aircraft, 69 images
A300 Total: 2 Train: 1 aircraft, 49 images /  Eval: 1 aircraft, 59 images
A319 Total: 17 Train: 12 aircraft, 887 images /  Eval: 5 aircraft, 245 images
A320 Total: 72 Train: 54 aircraft, 3045 images /  Eval: 18 aircraft, 1004 images
A321 Total: 27 Train: 20 aircraft, 1209 images /  Eval: 7 aircraft, 295 images
Bombardier Challenger Total: 10 Train: 7 aircraft, 351 images /  Eval: 3 aircraft, 137 images
CRJ550 Total: 2 Train: 1 aircraft, 51 images /  Eval: 1 aircraft, 59 images
CRJ700 Total: 5 Train: 3 aircraft, 161 images /  Eval: 2 aircraft, 46 images
CRJ900 Total: 9 Train: 6 aircraft, 323 image

### Add Detection Evaluation

In [25]:
view = dataset.match_tags("multi_class_eval")
print(view)

Dataset:     plane-dataset
Media type:  image
Num samples: 3447
Tags:        ['capture-3-29', 'capture-3-30', 'capture-5-13', 'eval', 'model_eval', 'multi_class_eval', 'train']
Sample fields:
    id:                                                                       fiftyone.core.fields.ObjectIdField
    filepath:                                                                 fiftyone.core.fields.StringField
    tags:                                                                     fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:                                                                 fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    external_id:                                                              fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    icao24:                                                                   fiftyone.core.fields.EmbeddedDocumentField(fiftyone

In [ ]:
for sample in view:
    if sample["detections"] == None:
        sample["detections"] = fo.Detections(detections=[])
        sample.save()

In [ ]:
# Evaluate `predictions` w.r.t. labels in `ground_truth` field
view = dataset.match_tags("eval")
print(view.first())
view.evaluate_detections(
    "881_40k_predict", gt_field="detections", eval_key="eval_881"
)



In [23]:
# Evaluate the detections in the `predictions` field with respect to the
# objects in the `ground_truth` field
view = dataset.match_tags("multi_class_eval")
print(view)
results = view.evaluate_detections(
    "multi_class_split_ssd_predict", gt_field="multi_class_detections", eval_key="eval_multi_split_ssd", compute_mAP=True
)





Dataset:     plane-dataset
Media type:  image
Num samples: 3447
Tags:        ['capture-3-29', 'capture-3-30', 'capture-5-13', 'eval', 'model_eval', 'multi_class_eval', 'train']
Sample fields:
    id:                                                                       fiftyone.core.fields.ObjectIdField
    filepath:                                                                 fiftyone.core.fields.StringField
    tags:                                                                     fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:                                                                 fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    external_id:                                                              fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    icao24:                                                                   fiftyone.core.fields.EmbeddedDocumentField(fiftyone

In [24]:
# Get the 10 most common classes in the dataset
counts = view.count_values("multi_class_detections.detections.label")
classes = sorted(counts, key=counts.get, reverse=True)[:20]

# Print a classification report for the top-10 classes
results.print_report(classes=classes)

# Print some statistics about the total TP/FP/FN counts
print("TP: %d" % dataset.sum("eval_multi_split_ssd_tp"))
print("FP: %d" % dataset.sum("eval_multi_split_ssd_fp"))
print("FN: %d" % dataset.sum("eval_multi_split_ssd_fn"))


print("mAP: {}".format(results.mAP()))
# 0.3957

plot = results.plot_pr_curves(classes=classes)
plot.show()

plot = results.plot_confusion_matrix(classes=classes)
plot.show()

                       precision    recall  f1-score   support

              ERJ-170       0.76      0.80      0.78       762
                 A320       0.63      0.84      0.72       669
              737-800       0.44      0.42      0.43       322
                 A319       0.55      0.04      0.08       245
                 A321       0.71      0.73      0.72       237
           Cessna Jet       0.53      0.77      0.63       221
              737-700       0.92      0.68      0.78       195
Bombardier Challenger       0.80      0.35      0.49       137
              737-900       0.33      0.12      0.18       113
              EMB-135       0.95      0.68      0.79       102
           Hawker 400       0.00      0.00      0.00        82
              777-200       0.93      0.96      0.94        69
           Hawker 800       0.55      0.10      0.16        62
               CRJ900       0.80      0.92      0.86        49
               CRJ700       0.92      0.48      0.63  

FigureWidget({
    'data': [{'hovertemplate': '<b>class: %{text}</b><br>recall: %{x}<br>precision: %{y}<extra>…

FigureWidget({
    'data': [{'mode': 'markers',
              'opacity': 0.1,
              'type': 'scatter',…

In [ ]:
session = fo.launch_app(dataset, auto=False)

# Convert to evaluation patches
eval_patches = view.to_evaluation_patches("eval")
print(eval_patches)

# View patches in the App
session.view = eval_patches

### Low confidence sample

In [ ]:
# Show samples with most false positives
# Evaluate `predictions` w.r.t. labels in `ground_truth` field
view = dataset.match_tags("eval")
print(view.first())
eval_view = (
    view
    .sort_by("auto_aug_eval_fp", reverse=True)
    .filter_labels("auto_aug_predict_tiled", F("auto_aug_eval") == "fp")
)
session.view = eval_view

### Fix Tags

In [52]:
eval_view = dataset.match_tags("multi_class_eval")
print(eval_view)
for sample in eval_view:
    try:
        sample.tags = list(filter(lambda x: x != "multi_class_eval", sample.tags))
        sample.save()
    except ValueError:
        pass

Dataset:     plane-dataset
Media type:  image
Num samples: 2519
Tags:        ['capture-3-29', 'capture-3-30', 'capture-5-13', 'eval', 'model_eval', 'model_train', 'multi_class_eval', 'train']
Sample fields:
    id:                                                                       fiftyone.core.fields.ObjectIdField
    filepath:                                                                 fiftyone.core.fields.StringField
    tags:                                                                     fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:                                                                 fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    external_id:                                                              fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    icao24:                                                                   fiftyone.core.fields.EmbeddedDocumen

In [53]:
norm_models = dataset.distinct("norm_model.label")
for norm_model in norm_models:
    view = dataset.filter_labels("norm_model", (F("label") == norm_model)).match(F("auto_aug_predict_tiled.detections").length()>0).match_tags("multi_class_train",bool=False)
    print("{}: {}".format(norm_model,len(view)))
    if len(view) >= 100:
        for sample in view:
            sample.tags.append("multi_class_eval")
            sample.save()

737-400: 46
737-700: 568
737-800: 1198
737-900: 567
757-200: 100
767-200: 40
777-200: 127
A300: 108
A319: 1032
A320: 3949
A321: 1404
Bombardier Challenger: 388
CRJ200: 44
CRJ550: 110
CRJ700: 107
CRJ900: 400
Cessna Jet: 799
EMB-135: 195
EMB-505: 117
ERJ-170: 3370
Falcon: 41
Gulfstream: 234
Hawker 400: 192
Hawker 800: 108
HondaJet: 45
King Air: 53
LearJet: 87
Learjet 45/60: 118
Pilatus PC-12: 187
Piper Cheyenne: 71


In [59]:
eval_view = dataset.match_tags("multi_class_eval")
print(eval_view)

Dataset:     plane-dataset
Media type:  image
Num samples: 15378
Tags:        ['capture-3-29', 'capture-3-30', 'capture-5-13', 'eval', 'model_eval', 'model_train', 'multi_class_eval', 'train']
Sample fields:
    id:                                                                       fiftyone.core.fields.ObjectIdField
    filepath:                                                                 fiftyone.core.fields.StringField
    tags:                                                                     fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:                                                                 fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    external_id:                                                              fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    icao24:                                                                   fiftyone.core.fields.EmbeddedDocume